This is an example of a jupyter notebook, running using the `pixi` environment we created to manage our dependencies. Note that you may need to change your python interpreter (top right corner) to:
`.pixi/envs/default/bin/python`

In [6]:
import os
import glob
import pandas as pd
import numpy as np
import random
import torch
import sklearn

from sklearn.metrics import roc_auc_score
from opensoundscape.ml import bioacoustics_model_zoo as bmz
from opensoundscape.ml.shallow_classifier import quick_fit 

Pre-processing

In [7]:
datapath = "/workspaces/non-avian-ml-toy/data/audio"
species = "bullfrog"
datatype = "data"

files = glob.glob(os.path.join(datapath, species, datatype, "**/*.wav"), recursive=True)
labels = pd.DataFrame({"file": files, "present": ["pos" in f.lower() for f in files]})
labels['file'] = labels['file'].astype(str)
labels.set_index("file", inplace=True)

pd.set_option('display.max_colwidth', 100)
print(labels.head())

                                                                                           present
file                                                                                              
/workspaces/non-avian-ml-toy/data/audio/bullfrog/data/pos/bullfrog-pos-t-11113588_154.wav     True
/workspaces/non-avian-ml-toy/data/audio/bullfrog/data/pos/bullfrog-pos-t-11113588_149.wav     True
/workspaces/non-avian-ml-toy/data/audio/bullfrog/data/pos/bullfrog-pos-t-11113588_145.wav     True
/workspaces/non-avian-ml-toy/data/audio/bullfrog/data/pos/bullfrog-pos-t-11113588_151.wav     True
/workspaces/non-avian-ml-toy/data/audio/bullfrog/data/pos/bullfrog-pos-t-11113588_152.wav     True


In [8]:
labels_train, labels_val = sklearn.model_selection.train_test_split(labels[['present']])

Run Models

In [9]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [12]:
# load a model from the model zoo
model = bmz.load('BirdNET') #or bmz.load('Perch')

emb_train = model.embed(labels_train, return_dfs=False, batch_size=128, num_workers=0)
emb_val = model.embed(labels_val, return_dfs=False, batch_size=128, num_workers=0)

classes = ['A']
birdnet.change_classes(classes) 

# fit the classification head with embeddings and labels
birdnet.network.fit(emb_train, labels_train.values, emb_val, labels_val.values)

# make predictions by passing the embeddings through the classifier
preds = birdnet.network(torch.tensor(emb_val)).detach()
# calculate the area under the ROC score
roc_auc_score(labels_val.values,preds,average=None)

Using cache found in /root/.cache/torch/hub/kitzeslab_bioacoustics-model-zoo_0.11.0.dev1


Exception: Failed to download file from url https://github.com/kahst/BirdNET-Analyzer/raw/main/labels/V2.4/BirdNET_GLOBAL_6K_V2.4_Labels_af.txt. Status code: 404

In [ ]:
preds = preds.detach().numpy()
plt.hist(preds[labels_val==True],bins=20,alpha=0.5,label='positives')
plt.hist(preds[labels_val==False],bins=20,alpha=0.5,label='negatives')
plt.legend()